## Check the setup and connect to the database

Check if your connection with HANA DB is still active.

In [1]:
%run 'exercise1-check_setup.ipynb'

SAP HANA Client for Python: 2.21.24062400
The file './00-setup/user.ini' exists.
Connection to SAP HANA db at a0f81553-537d-4e3a-808d-fe370ba4095a.hana.prod-eu20.hanacloud.ondemand.com:443 
will be attempted for the user INNOV8ION_USER2
Connected to SAP HANA db version 4.00.000.00.1732009081 (fa/CE2024.28) 
at a0f81553-537d-4e3a-808d-fe370ba4095a.hana.prod-eu20.hanacloud.ondemand.com:443 as user INNOV8ION_USER2
Current time on the SAP HANA server: 2024-12-12 14:12:28.322000


## Use HANA DataFrame and Pandas DataFrame

Check which tables are available in schema **DB1**. Table **TITANIC** contains all historical data including the **SURVIVED** column. **TITANIC_TEST** contains fictional testing data (without the **SURVIVED** column). **TITANIC_TRUTH** contains the same data as **TITANIC_TEST**, and it also contains the **SURVIDED** column.

In [2]:
myconn.get_tables(schema='DB_1')

,TABLE_NAME
0,TITANIC
1,TITANIC_TEST
2,TITANIC_TRUTH


A table with data already exist in your SAP HANA database, so you use [the `table()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_3_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.ConnectionContext.table) to create a HANA DataFrame from an existing database table. 

In [3]:
hdf_train=myconn.table('TITANIC', schema='DB_1')

You can always use [the `select_statement` property](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_3_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame) to check the SQL SELECT statement that backs a HANA DataFrame. 

In [4]:
hdf_train.select_statement

'SELECT * FROM "DB_1"."TITANIC"'

HANA DataFrame represents only SQL SELECT statement, but does not store data...

In [8]:
hdf_train_first10recs=hdf_train.head(10)

In [9]:
hdf_train_first10recs.select_statement

'SELECT TOP 10 * FROM (SELECT * FROM "DB_1"."TITANIC") dt'

...until [a `collect()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_3_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.collect) is executed, which returns a result as a Pandas dataframe on a client side. The data is from data science competition website Kaggle (https://www.kaggle.com/competitions/titanic). More info is available on the meaning of the columns etc. here: https://www.kaggle.com/competitions/titanic/data.

In [10]:
hdf_train_first10recs.collect()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",m,22.0,1,0,A/5 21171,7,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",f,38.0,1,0,PC 17599,71,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",f,26.0,0,0,STON/O2. 3101282,7,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",f,35.0,1,0,113803,53,C123,S
4,5,0,3,"Allen, Mr. William Henry",m,35.0,0,0,373450,8,None,S
5,6,0,3,"Moran, Mr. James",m,NaN,0,0,330877,8,None,Q
6,7,0,1,"McCarthy, Mr. Timothy J",m,54.0,0,0,17463,51,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",m,2.0,3,1,349909,21,None,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",f,27.0,0,2,347742,11,None,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",f,14.0,1,0,237736,30,None,C


You use [HANA `DataFrame` methods](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_3_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame) to query the data from SAP HANA database.

In [12]:
print(hdf_train.value_counts(['PCLASS']).select_statement)

SELECT T0."VALUES", T1."NUM_PCLASS"
FROM (SELECT DISTINCT "VALUES" FROM ((SELECT CAST("VALUES" AS NVARCHAR(255)) AS "VALUES" FROM (SELECT "PCLASS" AS "VALUES" FROM (SELECT "PCLASS" FROM (SELECT * FROM "DB_1"."TITANIC") AS "DT_3") AS "DT_6") AS "DT_7")) AS "DT_13") T0 LEFT OUTER JOIN (SELECT CAST("PCLASS" AS NVARCHAR(255)) AS "PCLASS", "NUM_PCLASS" FROM (SELECT "PCLASS", count("PCLASS") AS "NUM_PCLASS" FROM (SELECT * FROM "DB_1"."TITANIC") AS "DT_3" GROUP BY "PCLASS") AS "DT_9") T1
 ON T0."VALUES" = T1."PCLASS"



In [13]:
hdf_train.value_counts(['PCLASS']).collect()

,VALUES,NUM_PCLASS
0,2,184
1,1,216
2,3,491


In [17]:
print(hdf_train.value_counts(['PCLASS']).sort('NUM_PCLASS', desc=True).select_statement)

SELECT * FROM (SELECT T0."VALUES", T1."NUM_PCLASS"
FROM (SELECT DISTINCT "VALUES" FROM ((SELECT CAST("VALUES" AS NVARCHAR(255)) AS "VALUES" FROM (SELECT "PCLASS" AS "VALUES" FROM (SELECT "PCLASS" FROM (SELECT * FROM "DB_1"."TITANIC") AS "DT_3") AS "DT_46") AS "DT_47")) AS "DT_53") T0 LEFT OUTER JOIN (SELECT CAST("PCLASS" AS NVARCHAR(255)) AS "PCLASS", "NUM_PCLASS" FROM (SELECT "PCLASS", count("PCLASS") AS "NUM_PCLASS" FROM (SELECT * FROM "DB_1"."TITANIC") AS "DT_3" GROUP BY "PCLASS") AS "DT_49") T1
 ON T0."VALUES" = T1."PCLASS"
) AS "DT_55" ORDER BY "NUM_PCLASS" DESC


In [19]:
hdf_train.value_counts(['PCLASS']).sort('NUM_PCLASS', desc=True).collect()

,VALUES,NUM_PCLASS
0,3,491
1,1,216
2,2,184


You use [Pandas `DataFrame` and/or `Series`](https://pandas.pydata.org/docs/user_guide/10min.html#minutes-to-pandas) methods to query the data returned to a client as a result of the `collect()` method.

In [20]:
hdf_train.value_counts(['PCLASS']).collect().sort_values('NUM_PCLASS')

,VALUES,NUM_PCLASS
0,2,184
1,1,216
2,3,491


🤓 **Let's discuss**:
1. HANA DataFrames
2. Pandas DataFrames/Series